In [1]:
import pandas as pd

In [15]:
import numpy as np

In [117]:
def process_data(filename="test_resources/test_android.tsv", var_threshold=20):
    android_data = pd.read_csv(filename, delimiter=" ", header=None)
    android_data.columns = ["heading", "steps", "time"]
    android_data["round_time"] = (np.round(android_data["time"]/500, 0)*500).astype(np.int64)
    android_data["step_diff"] = android_data["steps"].diff()
    heading_bin_var = android_data.groupby("round_time")["heading"].var()
    heading_bin_mean = android_data.groupby("round_time")["heading"].mean()
    step_bin_max = android_data.groupby("round_time")["step_diff"].sum()
    step_bin_max.name = "steps"
    heading_bin_mean.name = "heading_mean"
    heading_bin_var.name = "heading_var"

    rounded_df = pd.concat([heading_bin_var, heading_bin_mean, step_bin_max], 1)
    rounded_df["turn"] = rounded_df["heading_var"] > var_threshold
    rounded_df["cum_turn"] = rounded_df["turn"].cumsum()
    grouped = rounded_df.groupby("cum_turn")
    headings = grouped["heading_mean"].median()
    start_step = grouped["steps"].min()
    end_step = grouped["steps"].max()
    steps = grouped["steps"].sum()

    instructions = pd.concat([headings, steps], 1)
    instructions.columns=["heading", "steps"]
    instructions["height"] = 0.0
    #instructions = instructions[instructions["steps"] > 0]
    return instructions

In [120]:
process_data(var_threshold=100).as_matrix(["steps", "heading", "height"])

array([[   6.        ,  122.23042511,    0.        ],
       [   3.        ,  118.09186659,    0.        ],
       [   1.        ,  136.29336915,    0.        ],
       [   8.        ,  279.36342529,    0.        ],
       [   1.        ,  243.87355575,    0.        ],
       [   2.        ,   32.67233785,    0.        ]])

In [103]:
android_data["step_diff"].sum()

21.0

In [81]:
instructions

,heading_mean,steps
cum_turn,,
1,125.193555,3
2,70.465754,2
3,118.091867,2
8,256.309735,1
9,264.682395,1
10,292.766928,1
14,54.202261,2
